In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
import re
from parse_membrane_ID import parse_mem_id


from glitch.impedance import EISSpectrumDoc, SingleSpectrum

INPUT_FOLDER = "/Users/andreakowal/Downloads/"
OUTPUT_FOLDER = "/Users/andreakowal/Coding/Raw Nyquist Data" # where the raw Nyquist data points are being saved
OUTPUT_INTERCEPTS = "/Users/andreakowal/Coding/Intercept Data" # where the x-intercept values are being saved

# data folder path
data_folder = Path(INPUT_FOLDER + "MTRON/40 mM CoCl2").expanduser()

# background correction files
OC_path = Path(INPUT_FOLDER + "MTRON/02_07_25_OCTest_C01.mpt").expanduser()
SC_path = Path(INPUT_FOLDER + "MTRON/02_03_25_SCTest_C01.mpt").expanduser()
oc = EISSpectrumDoc.from_eclab_mpt(OC_path)
sc = EISSpectrumDoc.from_eclab_mpt(SC_path)

# sort membrane files by membrane ID and replicate number
def extract_membrane_and_replicate (data_folder):
    membrane_ID = re.search(r"_(\d+)_R", data_folder.name) #Membrane number is after the first underscore
    replicate = re.search(r"_R(\d+)_", data_folder.name) #Replicate number is after the R and before 3rd underscore

    membrane_number = int(membrane_ID.group(1)) if membrane_ID else "Membrane ID not found"
    replicate_number = int(replicate.group(1)) if replicate else "Replicate not found"
    return (membrane_number, replicate_number)

nyquist_results = []

def trendline_intercept(Z_real, Z_imag):
    slope, intercept = np.polyfit(Z_real, Z_imag, 1)
    return -intercept / slope

for file_path in sorted(data_folder.glob("*.mpt"), key = extract_membrane_and_replicate):
    print(f"Processing {file_path.name}")

    my_spectrum = EISSpectrumDoc.from_eclab_mpt(file_path)
    my_spectrum.background_correct(oc.cycles_raw[0], sc.cycles_raw[0])

    # filtering raw data
    raw_data = my_spectrum.cycles_raw[0]
    mask = raw_data.frequencies <= 250_000
    freqs_filtered = raw_data.frequencies[mask]
    impedance_filtered = raw_data.impedance[mask]

    Z_real = impedance_filtered.real
    Z_imag = impedance_filtered.imag

    df_points = pd.DataFrame({
        "Frequency (Hz)": freqs_filtered,
        "Z_real (Ohm)": Z_real,
        "Z_imag (Ohm)": Z_imag})

    # save raw Nyquist CVS files to output folder
    output_folder = Path(OUTPUT_FOLDER)
    output_filename = output_folder /f"{file_path.stem}_points.csv"
    df_points.to_csv(output_filename, index=False)
    print(f"Saved data table to: {output_filename}")

    # Parse ID for meta data
    id_string = file_path.stem
    print (id_string)
    meta_data = parse_mem_id(id_string)
    print (meta_data)

    # Creating cleaner title for graphs
    graph_title = f"{meta_data['Soak Concentration (M)']} {meta_data['Salt']}"

    # Build subtitles
    subtitle_parts = []

    if "Dip Concentration (M)" in meta_data:
        subtitle_parts.append(f"{meta_data['Dip Concentration (M)']} dip")

    subtitle_parts.append(f"Membrane: {meta_data['Membrane']}")
    subtitle_parts.append(f"ID: {meta_data['Membrane ID']}")
    subtitle_parts.append(f"Replicate: {meta_data['Replicate']}")
    subtitle_parts.append(f"Date: {meta_data["Date"]}")

    subtitle = " | ".join(subtitle_parts)
    
    combined_title = f"{graph_title}<br><span style='font-size:14px'>{subtitle}</span>"

    # Create trendline
    trendline = np.poly1d(np.polyfit(Z_real, Z_imag, 1))
    Z_imag_trend = trendline(Z_real)

    # Plotly Nyquist plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=Z_real,
        y=Z_imag, 
        mode='markers',
        name='EIS Data',
        marker=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=Z_real,
        y=Z_imag_trend,
        mode='lines',
        name='Trendline',
        line=dict(color='black')
    ))

    fig.update_layout(
        title= combined_title,
        xaxis_title='Re{Z} (Ohm)',
        yaxis_title='-Im{Z} (Ohm)',
        xaxis=dict(scaleanchor="y", scaleratio=1),  # equal aspect ratio
        yaxis=dict(),
        width=700,
        height=600
    )

    fig.show()

    filtered_cycle = SingleSpectrum(frequencies=freqs_filtered, impedance=impedance_filtered)
    filtered_doc = EISSpectrumDoc(cycles=[filtered_cycle])

    x_intercept = trendline_intercept(Z_real, Z_imag)

    nyquist_results.append({"filename": file_path.name, "real intercept": x_intercept})

df = pd.DataFrame(nyquist_results)
summary_file = f"/EIS_summary_{meta_data['Soak Concentration (M)'].replace(' ', '')}_{meta_data['Salt'].replace(' ', '_')}.csv"
df.to_csv(OUTPUT_INTERCEPTS + summary_file, index=False)
print(f"Saved summary file to: {OUTPUT_INTERCEPTS + summary_file}")
print(df)



Processing 40mMCoCl2_M02_R1_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M02_R1_FKS50_20250424_C01_points.csv
40mMCoCl2_M02_R1_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '02', 'Replicate': '1', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M01_R1_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M01_R1_FKS50_20250424_C01_points.csv
40mMCoCl2_M01_R1_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '01', 'Replicate': '1', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M03_R1_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M03_R1_FKS50_20250424_C01_points.csv
40mMCoCl2_M03_R1_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '03', 'Replicate': '1', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M02_R2_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M02_R2_FKS50_20250424_C01_points.csv
40mMCoCl2_M02_R2_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '02', 'Replicate': '2', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M01_R2_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M01_R2_FKS50_20250424_C01_points.csv
40mMCoCl2_M01_R2_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '01', 'Replicate': '2', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M03_R2_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M03_R2_FKS50_20250424_C01_points.csv
40mMCoCl2_M03_R2_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '03', 'Replicate': '2', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M02_R3_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M02_R3_FKS50_20250424_C01_points.csv
40mMCoCl2_M02_R3_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '02', 'Replicate': '3', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M03_R3_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M03_R3_FKS50_20250424_C01_points.csv
40mMCoCl2_M03_R3_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '03', 'Replicate': '3', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Processing 40mMCoCl2_M01_R3_FKS50_20250424_C01.mpt
Saved data table to: /Users/andreakowal/Coding/Raw Nyquist Data/40mMCoCl2_M01_R3_FKS50_20250424_C01_points.csv
40mMCoCl2_M01_R3_FKS50_20250424_C01
{'Salt': 'CoCl2', 'Membrane': 'FKS50', 'Membrane ID': '01', 'Replicate': '3', 'Soak Concentration (M)': '40m M', 'Date': '04/24/2025'}


Saved summary file to: /Users/andreakowal/Coding/Intercept Data/EIS_summary_40mM_CoCl2.csv
                                  filename  real intercept
0  40mMCoCl2_M02_R1_FKS50_20250424_C01.mpt        1.047883
1  40mMCoCl2_M01_R1_FKS50_20250424_C01.mpt        1.025109
2  40mMCoCl2_M03_R1_FKS50_20250424_C01.mpt        1.018335
3  40mMCoCl2_M02_R2_FKS50_20250424_C01.mpt        1.115909
4  40mMCoCl2_M01_R2_FKS50_20250424_C01.mpt        1.090343
5  40mMCoCl2_M03_R2_FKS50_20250424_C01.mpt        1.154923
6  40mMCoCl2_M02_R3_FKS50_20250424_C01.mpt        1.159176
7  40mMCoCl2_M03_R3_FKS50_20250424_C01.mpt        1.385944
8  40mMCoCl2_M01_R3_FKS50_20250424_C01.mpt        1.097401
